In [104]:
#Load libraries
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import Lasso, LogisticRegression
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error


In [87]:
#Load data
url = 'demographic_health_data.csv'
df_raw = pd.read_csv(url)
df_raw.info(max_cols=110)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Data columns (total 108 columns):
 #    Column                                                                         Non-Null Count  Dtype  
---   ------                                                                         --------------  -----  
 0    fips                                                                           3140 non-null   int64  
 1    TOT_POP                                                                        3140 non-null   int64  
 2    0-9                                                                            3140 non-null   int64  
 3    0-9 y/o % of total pop                                                         3140 non-null   float64
 4    19-Oct                                                                         3140 non-null   int64  
 5    10-19 y/o % of total pop                                                       3140 non-null   float64
 6    20-29         

In [88]:
df_raw.sample(10, random_state=2025)

,fips,TOT_POP,0-9,0-9 y/o % of total pop,19-Oct,10-19 y/o % of total pop,20-29,20-29 y/o % of total pop,30-39,30-39 y/o % of total pop,...,COPD_number,diabetes_prevalence,diabetes_Lower 95% CI,diabetes_Upper 95% CI,diabetes_number,CKD_prevalence,CKD_Lower 95% CI,CKD_Upper 95% CI,CKD_number,Urban_rural_code
120,5023,24965,2423,9.705588,2757,11.043461,2332,9.341078,2541,10.178250,...,2302,14.6,13.3,15.8,2946,3.9,3.6,4.2,790,6
686,17187,17032,2047,12.018553,2427,14.249648,2193,12.875763,1901,11.161343,...,1175,11.5,10.5,12.5,1525,3.2,3.0,3.5,430,6
3098,55105,163129,19981,12.248589,21849,13.393695,19982,12.249202,20353,12.476629,...,8680,10.4,9.5,11.2,13020,3.0,2.8,3.3,3784,4
1512,29063,12630,1193,9.445764,1189,9.414093,1835,14.528899,1942,15.376089,...,984,12.0,11.1,13.0,1258,3.1,2.9,3.3,322,4
2836,51036,6941,536,7.722230,636,9.162945,734,10.574845,670,9.652788,...,600,17.7,16.7,18.7,1043,4.3,4.1,4.6,254,2
4,1009,57840,7095,12.266598,7570,13.087828,6742,11.656293,6884,11.901798,...,4661,13.5,12.6,14.5,6017,3.4,3.2,3.7,1507,2
2559,48077,10456,1041,9.956006,1271,12.155700,998,9.544759,1070,10.233359,...,771,13.5,12.3,14.6,1125,3.5,3.3,3.8,294,4
2727,48413,2895,339,11.709845,490,16.925734,293,10.120898,384,13.264249,...,154,14.2,13.3,15.1,306,3.4,3.2,3.7,74,6
2311,44005,82542,7243,8.774927,9194,11.138572,10520,12.745027,9527,11.542003,...,4598,9.3,8.4,10.4,6410,2.8,2.6,3.1,1922,2
1306,26155,68192,7384,10.828250,8728,12.799155,8258,12.109925,7597,11.140603,...,5037,11.0,10.1,11.9,5936,3.1,2.9,3.4,1682,5


In [89]:
df_baking = df_raw.copy()
df_baking.columns = df_baking.columns.str.lower()
df = df_baking.copy()
df.info(max_cols=110)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3140 entries, 0 to 3139
Data columns (total 108 columns):
 #    Column                                                                         Non-Null Count  Dtype  
---   ------                                                                         --------------  -----  
 0    fips                                                                           3140 non-null   int64  
 1    tot_pop                                                                        3140 non-null   int64  
 2    0-9                                                                            3140 non-null   int64  
 3    0-9 y/o % of total pop                                                         3140 non-null   float64
 4    19-oct                                                                         3140 non-null   int64  
 5    10-19 y/o % of total pop                                                       3140 non-null   float64
 6    20-29         

In [90]:
#EDA
df_train, df_test = train_test_split(df, test_size=0.1, random_state=2025)
df_train = df_train.reset_index(drop=True)
df_test = df_test.reset_index(drop=True)
df_train.shape, df_test.shape

((2826, 108), (314, 108))

In [91]:
display(df_train.describe(include='number').T)
display(df_train.describe(include='object').T)

,count,mean,std,min,25%,50%,75%,max
fips,2826.0,30386.630219,15146.443690,1003.0,19001.500000,29168.000000,45080.000000,5.604500e+04
tot_pop,2826.0,106297.384996,345553.194053,88.0,10980.500000,26148.500000,69560.750000,1.010552e+07
0-9,2826.0,13021.275655,43342.182304,0.0,1281.000000,3086.000000,8350.500000,1.208253e+06
0-9 y/o % of total pop,2826.0,11.881376,2.120271,0.0,10.623856,11.805865,12.959731,2.546068e+01
19-oct,2826.0,13651.469922,43819.022626,0.0,1373.000000,3299.000000,9122.500000,1.239139e+06
...,...,...,...,...,...,...,...,...
ckd_prevalence,2826.0,3.437933,0.570028,1.9,3.100000,3.400000,3.800000,6.200000e+00
ckd_lower 95% ci,2826.0,3.199752,0.529588,1.8,2.800000,3.200000,3.500000,5.800000e+00
ckd_upper 95% ci,2826.0,3.702052,0.615501,2.0,3.300000,3.700000,4.100000,6.600000e+00
ckd_number,2826.0,2505.172328,7983.118501,3.0,315.750000,721.000000,1791.500000,2.377660e+05


,count,unique,top,freq
county_name,2826,1697,Washington,29
state_name,2826,51,Texas,222


In [100]:
#ML
X_train = df_train.drop(columns=['heart disease_number','county_name','state_name'])
y_train = df_train['heart disease_number']

X_test = df_test.drop(columns=['heart disease_number','county_name','state_name'])
y_test = df_test['heart disease_number']

In [102]:
model = LogisticRegression()

In [103]:
model.fit(X_train, y_train)
y_hat = model.predict(X_test)

/workspaces/linear_regression2/.venv/lib/python3.12/site-packages/sklearn/utils/multiclass.py:213: UserWarning: The number of unique classes is greater than 50% of the number of samples. `y` could represent a regression problem, not a classification problem.
  y_type = type_of_target(y, input_name="y")
/workspaces/linear_regression2/.venv/lib/python3.12/site-packages/sklearn/linear_model/_logistic.py:473: ConvergenceWarning: lbfgs failed to converge after 100 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase the number of iterations to improve the convergence (max_iter=100).
You might also want to scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


In [106]:
print(f'MSE: {mean_squared_error(y_test, y_hat)}')
print(f'MAE: {mean_absolute_error(y_test, y_hat)}')
print(f'RMSE: {np.sqrt(mean_squared_error(y_test, y_hat))}')
print(f'R2: {r2_score(y_test, y_hat)}')

MSE: 39032481.23248408
MAE: 1820.5
RMSE: 6247.598037044643
R2: 0.6873278151582611


In [107]:
alpha = 1.0
lasso_model = Lasso(alpha = alpha)

lasso_model.fit(X_train, y_train)

score = lasso_model.score(X_test, y_test)
print("Coefficients:", lasso_model.coef_)
print("R2 score:", score)

Coefficients: [-4.62471780e-02  2.36470465e-02 -6.04301416e-04  1.67364366e+01
 -6.37902144e-02  3.57523032e+01 -1.06629491e-02  6.10546012e+01
 -6.43695361e-03  2.44450404e+01  1.15829442e-01 -9.19587085e-01
  4.58539305e-02  2.43622829e+01 -4.74298353e-02  2.94976323e+01
  1.40287701e-01 -5.17870853e+01  1.53057887e-01 -3.59952743e+01
 -2.42266239e-02  1.72085010e+01 -2.59254594e-02  2.02009143e+01
 -1.33305238e-02  1.02763540e+01 -3.71706076e-02  5.36636194e+01
  6.03689766e-03  1.01906741e+01 -2.99352623e-02  0.00000000e+00
  6.46624165e-03  7.08606298e-02  1.51932938e-03  0.00000000e+00
  3.28401622e+00 -2.59775514e+00  1.24955196e+01 -3.31488932e+00
 -0.00000000e+00  7.86862599e-03  1.80455424e-02 -1.66117213e-02
  1.19969917e-03  6.32021297e+00 -5.22622608e+00  9.33493462e+00
 -4.47074500e+00  6.12737605e-02 -2.60991116e+01  6.68158061e+00
 -3.42248905e+00 -5.89744317e-02  2.35489117e-03 -4.33984859e-03
 -7.17853873e-03  4.46010967e-04 -2.50572027e-01  5.59723852e+01
  5.8921739

/workspaces/linear_regression2/.venv/lib/python3.12/site-packages/sklearn/linear_model/_coordinate_descent.py:695: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.203e+08, tolerance: 7.261e+07
  model = cd_fast.enet_coordinate_descent(
